Conversion required:
* In the forward processing below, the eigenvector is 8-bit and the input is ternary but represented in 8-bits. But the result may be bit_depth_max\*L - actually as the eigenvector is normalised this will be int(1/sqrt(L))\*127)\*L.
* In the backward processing, processed_r_back is ternary (but stored as 8-bit) and eigenvector is 8-bit as above. The result is either -1\*eigenvector, 0 or eigenvector.
* We can likely implement all casting control in the project method.

In [1]:
import numpy as np

In [149]:
from src.var_processor.covariance8bit import CovarianceUnit
from src.var_processor.power_iterator8bit import PowerIterator
from src.var_processor.pb_threshold import ternary_pbt

def project(vec_1, vec_2):
    """Project input using eigenvector.

    Args:
        vec1: 1D numpy array.
        vec2: 1D numpy array.
    """
    return np.dot(vec_1, vec_2)


class VPU:
    """Variance processing unit."""

    def __init__(self, size):
        """Initialise.

        Args:
            size: integer setting the 1D size of an input;
        """
        self.cu = CovarianceUnit(size)
        self.pi = PowerIterator(size)
        self.size = size

    def forward_processing(self, forward_data):
        """Process data to apply to forward input data."""
        return forward_data

    def pred_input_processing(self, pred_inputs):
        """Process data to apply to output predicted inputs."""
        return pred_inputs

    def r_forward_processing(self, r_forward):
        """Process data to apply to output r_forward value."""
        return r_forward

    def r_backward_processing(self, r_backward):
        """Process data to apply to output r_forward value."""
        return r_backward

    def iterate(self, forward_data, r_backward):
        """Iterate through one discrete timestep.

        Args:
            forward_data: data for feedforward transformation
                1D numpy array of length self.size.
            r_backward: scalar value indicating a prediction of r.

        Returns:
            pred_inputs: 1D array containing the predicted input
            r: scalar feature detection output

        """
        r_forward = self.forward(forward_data)
        pred_inputs = self.backward(r_backward)
        return r_forward, pred_inputs

    def forward(self, forward_data):
        """Forward pass to generate cause - r.

        Args:
            forward_data: 1D numpy array of length self.size.
            This is the residual data rather than the original data.
        Returns:
            r_forward: scalar feature detection output

        """
        # Perform optional pre-processing
        processed_data = self.forward_processing(forward_data)
        # Project
        r_forward = project(self.pi.eigenvector.T, processed_data)
        # Perform optional post-processing
        processed_output = self.r_forward_processing(r_forward)
        return processed_output

    def backward(self, r_backward):
        """Backward pass to generate predicted inputs.

        The predicted inputs are the original not residual inputs.

        Args:
            r_backward: scalar cause feedback.
        Returns:
            pred_inputs: numpy array of predicted inputs of size - size.

        """
        # Perform optional pre-processing
        processed_r_back = self.r_backward_processing(r_backward)
        # Use item to convert r to scalar
        pred_inputs = project(processed_r_back.item(), self.pi.eigenvector)
        # Perform optional post-processing
        processed_output = self.pred_input_processing(pred_inputs)
        return processed_output

    def update_cov(self, input_data, power_iterate=False):
        """Update the covariance matrix.

        Use this to bed in the covariance.

        Args:
            input_data: 1D numpy array of length self.size.
            This is the original rather than residual data.
        """
        self.cu.update_cov(input_data)
        if power_iterate:
            cov = self.cu.covariance
            # Power iterate
            self.pi.load_covariance(cov)
            self.pi.iterate()

    def reset(self):
        """Reset and clear."""
        self.__init__(self.size)
        
    def __repr__(self):
        """String representation of class."""
        string = (
            "\n-----\n"
            f"VPU of length {self.size}\n"
            "\n-----\n"
            f"Power Iterator: {self.pi.__repr__()}\n"
            "\n-----\n"
            f"Covariance:{self.cu.__repr__()}\n"
            "\n-----\n"
            "\n-----\n"
        )
        return string

### Perform Original VPU Tests on Above 

We'll now get integer values.

In [150]:
from src.tests.test_vpu import rand_same, rand_diff, rand_opposite

In [151]:
vpu = VPU(2)
for _ in range(0, 1000):
    vpu.update_cov(rand_same(), power_iterate=True)
# Check all values of covariance matrix are the same
print(vpu.cu.covariance, vpu.cu.covariance/127)
print(vpu.pi.eigenvector, (1/np.sqrt(2))*127)

[[63 63]
 [63 63]] [[0.49606299 0.49606299]
 [0.49606299 0.49606299]]
[[-89]
 [-89]] 89.80256121069152


In [152]:
vpu.forward(ones)

ValueError: shapes (1,2) and (4,1) not aligned: 2 (dim 1) != 4 (dim 0)

In [153]:
ones = np.ones(shape=(2,1), dtype=np.int8); ones

array([[1],
       [1]], dtype=int8)

Overflow.

In [154]:
processed_data = ones*(np.sqrt(2)/2)
project(vpu.pi.eigenvector.T, processed_data).astype(np.int8)

array([[-125]], dtype=int8)

In [155]:
odd = np.asarray([[1], [0]]); odd

processed_data = odd*(np.sqrt(2)/2)
project(vpu.pi.eigenvector.T, processed_data).astype(np.int8)

array([[-62]], dtype=int8)

Scaling works.

In [156]:
vpu.pi.eigenvalue

array([126], dtype=uint16)

In [157]:
sample_1 = np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue)); sample_1

array([-999.02246, -999.02246], dtype=float32)

In [158]:
sample_1 = np.dot(vpu.pi.eigenvector/127, np.sqrt(vpu.pi.eigenvalue/127)); sample_1

array([-0.69802294, -0.69802294])

In [159]:
vpu = VPU(2)
for _ in range(0, 1000):
    vpu.update_cov(rand_same(negative=True), power_iterate=True)
# Check all values of covariance matrix are the same
print(vpu.cu.covariance, vpu.cu.covariance/127)
print(vpu.pi.eigenvector, (1/np.sqrt(2))*127)

[[81 81]
 [81 81]] [[0.63779528 0.63779528]
 [0.63779528 0.63779528]]
[[-89]
 [-89]] 89.80256121069152


In [160]:
np.linalg.norm(vpu.pi.eigenvector)

125.86500705120545

In [161]:
vpu.pi.eigenvalue

array([162], dtype=uint16)

In [162]:
sample_1 = np.dot(vpu.pi.eigenvector/127, np.sqrt(vpu.pi.eigenvalue/127)); sample_1

array([-0.79148362, -0.79148362])

In [163]:
size = 4
vpu = VPU(size)
for _ in range(0, 1000):
    vpu.update_cov(rand_same(negative=True, size=size), power_iterate=True)
# Check all values of covariance matrix are the same
print(vpu.cu.covariance, vpu.cu.covariance/127)
print(vpu.pi.eigenvector, (np.sqrt(4)/4)*127)

[[82 82 82 82]
 [82 82 82 82]
 [82 82 82 82]
 [82 82 82 82]] [[0.64566929 0.64566929 0.64566929 0.64566929]
 [0.64566929 0.64566929 0.64566929 0.64566929]
 [0.64566929 0.64566929 0.64566929 0.64566929]
 [0.64566929 0.64566929 0.64566929 0.64566929]]
[[-63]
 [-63]
 [-63]
 [-63]] 63.5


In [164]:
vpu.pi.eigenvector

array([[-63],
       [-63],
       [-63],
       [-63]], dtype=int8)

In [165]:
size = 4
ones = np.ones(shape=(size,1), dtype=np.int8); print(ones)
processed_data = ones*(np.sqrt(size)/size); print(processed_data)
print(project(vpu.pi.eigenvector.T, processed_data))
print(project(vpu.pi.eigenvector.T, processed_data).astype(np.int8))

[[1]
 [1]
 [1]
 [1]]
[[0.5]
 [0.5]
 [0.5]
 [0.5]]
[[-126.]]
[[-126]]


In [166]:
size = 4
neg_ones = -1*np.ones(shape=(size,1), dtype=np.int8); print(neg_ones)
processed_data = neg_ones*(np.sqrt(size)/size); print(processed_data)
print(project(vpu.pi.eigenvector.T, processed_data))
print(project(vpu.pi.eigenvector.T, processed_data).astype(np.int8))

[[-1]
 [-1]
 [-1]
 [-1]]
[[-0.5]
 [-0.5]
 [-0.5]
 [-0.5]]
[[126.]]
[[126]]


Ah - so this should be reversed sign and we are getting overflow. Our eigenvectors are one integer too big at -64 - should be -63. On the positive side we are still fine and have 63.

We can now PBT with max value = 126.

In [167]:
-127//10.9, -127/10.9, 127//10.9, 127/10.9

(-12.0, -11.65137614678899, 11.0, 11.65137614678899)

Ah that is why - integer rounds down for negative numbers. We need to adjust our normalise function. **FIXED** (By taking the sign out.

#### Test Different and Opposite

In [168]:
size = 2
vpu = VPU(size)
for _ in range(0, 1000):
    vpu.update_cov(rand_diff(size=size), power_iterate=True)
# Check all values of covariance matrix are the same
print(vpu.cu.covariance, vpu.cu.covariance/127)
print(vpu.pi.eigenvector)

[[72  0]
 [ 0 55]] [[0.56692913 0.        ]
 [0.         0.43307087]]
[[-127]
 [   0]]


In [169]:
test_array = np.asarray([[1], [0]])
processed_data = test_array*(np.sqrt(2)/2); print(processed_data)
r = vpu.forward(processed_data).astype(np.int8)
print(r)

[[0.70710678]
 [0.        ]]
[[-89]]


In [170]:
test_array = np.asarray([[1], [0]])
processed_data = test_array; print(processed_data)
r = vpu.forward(processed_data).astype(np.int8)
print(r)

[[1]
 [0]]
[[-127]]


So we want to scale conditional on the values. Something like sum(abs(array)) - will be 1 if one entry, 2 or size if entry. 

**Ah we actually want to scale by sqrt(sum(squares))/non_zeros i.e. the length of the input. To make out input unit length. **

But as we have ternary values we don't need to square - just take the absolute.

We did have:
```
self.scale_forward = np.sqrt(self.size)/self.size
self.scale_backward = self.size/np.sqrt(self.size)
```

In [171]:
size = 2
ones = np.ones(shape=(size,1), dtype=np.int8); print(ones)
print(ones/np.linalg.norm(ones))
size = 4
ones = np.ones(shape=(size,1), dtype=np.int8); print(ones)
print(ones/np.linalg.norm(ones))

[[1]
 [1]]
[[0.70710678]
 [0.70710678]]
[[1]
 [1]
 [1]
 [1]]
[[0.5]
 [0.5]
 [0.5]
 [0.5]]


In [172]:
%%timeit
np.linalg.norm(ones)

8.1 µs ± 167 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [173]:
%%timeit
np.sqrt(np.abs(ones).sum())

8.78 µs ± 352 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [174]:
print(np.linalg.norm(ones), np.sqrt(np.abs(ones).sum()))

2.0 2.0


In [175]:
np.linalg.norm(4)

4.0

In [176]:
np.linalg.norm(vpu.pi.eigenvector)

127.0

We can either divide the ones by the length of the vector or multiply then divide. If we divide before we can keep the projection in 8-bit space. Or we can project in 16-bit space and divide by the norm and then convert back to 8-bit. Depends if the division takes longer than the casting.

What about the backward processing?

Do we scale in anyway? We can multiply by the norm? But the norm is one. We need to multiply by the norm of the vector that is output! How do we do this before the vector is generated?

In [177]:
vpu.pi.eigenvector

array([[-127],
       [   0]], dtype=int8)

In [208]:
class BinaryVPU(VPU):
    """Let's update our functions modularly."""
        
    def forward_processing(self, forward_data):
        """Process data to apply to forward input data."""
        if forward_data.any():
            forward_data = forward_data*127//np.linalg.norm(forward_data)
        return forward_data
    
    def r_forward_processing(self, r_forward):
        """Scale r to -1 to 1 and PBT."""
        # Threshold r
        r_forward = r_forward//127
        pbt_output = ternary_pbt(r_forward, 127)
        return pbt_output
    

In [240]:
size = 2
vpu = BinaryVPU(size)
for _ in range(0, 1000):
    vpu.update_cov(rand_diff(size=size), power_iterate=True)

print(vpu.pi.eigenvector)

[[127]
 [  0]]


In [241]:
for i in range(0, 100):
    rand_input = np.random.randint(low=-1, high=2, size=(2, 1))
    r = vpu.forward(rand_input)
    print(rand_input, r, "\n\n")

[[-1]
 [-1]] [[-1]] 


[[0]
 [1]] [[0]] 


[[-1]
 [ 0]] [[-1]] 


[[0]
 [0]] [[0]] 


[[ 1]
 [-1]] [[1]] 


[[0]
 [1]] [[0]] 


[[1]
 [0]] [[1]] 


[[-1]
 [ 1]] [[-1]] 


[[1]
 [0]] [[1]] 


[[-1]
 [-1]] [[-1]] 


[[ 1]
 [-1]] [[1]] 


[[-1]
 [ 1]] [[-1]] 


[[-1]
 [ 0]] [[-1]] 


[[ 0]
 [-1]] [[0]] 


[[0]
 [0]] [[0]] 


[[ 1]
 [-1]] [[1]] 


[[-1]
 [ 0]] [[-1]] 


[[-1]
 [ 0]] [[-1]] 


[[0]
 [0]] [[0]] 


[[0]
 [1]] [[0]] 


[[ 1]
 [-1]] [[0]] 


[[-1]
 [ 1]] [[0]] 


[[ 1]
 [-1]] [[0]] 


[[0]
 [1]] [[0]] 


[[-1]
 [ 0]] [[-1]] 


[[-1]
 [ 0]] [[-1]] 


[[ 0]
 [-1]] [[0]] 


[[-1]
 [ 0]] [[-1]] 


[[ 0]
 [-1]] [[0]] 


[[ 0]
 [-1]] [[0]] 


[[-1]
 [-1]] [[-1]] 


[[-1]
 [ 0]] [[-1]] 


[[0]
 [1]] [[0]] 


[[-1]
 [ 1]] [[0]] 


[[ 1]
 [-1]] [[1]] 


[[ 0]
 [-1]] [[0]] 


[[-1]
 [ 0]] [[-1]] 


[[0]
 [1]] [[0]] 


[[-1]
 [ 1]] [[0]] 


[[-1]
 [ 0]] [[-1]] 


[[1]
 [0]] [[1]] 


[[-1]
 [-1]] [[-1]] 


[[ 0]
 [-1]] [[0]] 


[[-1]
 [ 0]] [[-1]] 


[[1]
 [0]] [[1]] 


[[ 0]
 [-1]] [[0]] 

In [242]:
size = 4
vpu = BinaryVPU(size)
for _ in range(0, 1000):
    vpu.update_cov(rand_diff(size=size), power_iterate=True)
print(vpu.pi.eigenvector)
for i in range(0, 100):
    rand_input = np.random.randint(low=-1, high=2, size=(size, 1))
    r = vpu.forward(rand_input)
    print(rand_input, r, "\n\n")

[[  0]
 [127]
 [  0]
 [  0]]
[[1]
 [0]
 [0]
 [1]] [[0]] 


[[ 0]
 [-1]
 [ 1]
 [-1]] [[-1]] 


[[ 1]
 [ 1]
 [-1]
 [ 1]] [[1]] 


[[ 1]
 [-1]
 [ 1]
 [-1]] [[-1]] 


[[ 0]
 [ 0]
 [-1]
 [ 1]] [[0]] 


[[ 0]
 [ 0]
 [ 0]
 [-1]] [[0]] 


[[-1]
 [ 1]
 [ 1]
 [ 1]] [[1]] 


[[ 0]
 [ 1]
 [ 0]
 [-1]] [[1]] 


[[ 1]
 [ 1]
 [-1]
 [ 0]] [[0]] 


[[-1]
 [ 1]
 [ 1]
 [-1]] [[1]] 


[[ 1]
 [ 0]
 [ 1]
 [-1]] [[0]] 


[[ 1]
 [ 0]
 [ 0]
 [-1]] [[0]] 


[[1]
 [1]
 [0]
 [0]] [[1]] 


[[-1]
 [-1]
 [ 0]
 [-1]] [[0]] 


[[0]
 [1]
 [0]
 [0]] [[1]] 


[[0]
 [1]
 [1]
 [0]] [[1]] 


[[1]
 [1]
 [0]
 [0]] [[0]] 


[[ 1]
 [ 0]
 [ 1]
 [-1]] [[0]] 


[[-1]
 [ 1]
 [-1]
 [ 1]] [[1]] 


[[ 0]
 [ 1]
 [ 0]
 [-1]] [[0]] 


[[ 1]
 [-1]
 [-1]
 [-1]] [[0]] 


[[ 0]
 [ 1]
 [ 0]
 [-1]] [[1]] 


[[-1]
 [-1]
 [ 1]
 [-1]] [[-1]] 


[[-1]
 [-1]
 [ 1]
 [-1]] [[0]] 


[[1]
 [0]
 [1]
 [0]] [[0]] 


[[ 1]
 [ 1]
 [-1]
 [-1]] [[1]] 


[[-1]
 [-1]
 [-1]
 [ 0]] [[-1]] 


[[-1]
 [ 0]
 [ 0]
 [-1]] [[0]] 


[[ 0]
 [-1]
 [ 1]
 [-1]] [[0]] 


[[1]


In [246]:
size = 4
vpu = BinaryVPU(size)
for _ in range(0, 1000):
    vpu.update_cov(rand_same(size=size), power_iterate=True)
print(vpu.pi.eigenvector)
try:
    for i in range(0, 100):
        rand_input = np.random.randint(low=-1, high=2, size=(size, 1))
        r = vpu.forward(rand_input)
        assert r in [-1, 0, 1]
        print(rand_input, r, "\n\n")
except AssertionError:
    print(f"Error: {vpu}\n\nInput: {rand_input}\nR: {r}\n\n")

[[63]
 [63]
 [63]
 [63]]
[[ 0]
 [-1]
 [-1]
 [ 0]] [[0]] 


[[0]
 [1]
 [0]
 [1]] [[1]] 


[[1]
 [1]
 [0]
 [1]] [[1]] 


[[-1]
 [ 0]
 [-1]
 [ 0]] [[0]] 


[[-1]
 [-1]
 [-1]
 [ 0]] [[0]] 


[[0]
 [1]
 [0]
 [0]] [[1]] 


[[-1]
 [ 1]
 [ 0]
 [ 1]] [[0]] 


[[ 0]
 [-1]
 [ 1]
 [ 0]] [[0]] 


[[1]
 [0]
 [0]
 [0]] [[0]] 


[[1]
 [0]
 [0]
 [0]] [[1]] 


[[ 0]
 [ 0]
 [-1]
 [ 0]] [[0]] 


[[0]
 [0]
 [0]
 [1]] [[0]] 


[[0]
 [0]
 [1]
 [1]] [[0]] 


[[ 0]
 [ 1]
 [-1]
 [ 0]] [[0]] 


[[ 0]
 [-1]
 [ 1]
 [ 1]] [[0]] 


[[ 0]
 [-1]
 [ 1]
 [ 0]] [[0]] 


[[-1]
 [-1]
 [-1]
 [-1]] [[-1]] 


[[ 1]
 [ 1]
 [-1]
 [ 1]] [[0]] 


[[ 1]
 [-1]
 [ 0]
 [ 0]] [[0]] 


[[ 1]
 [ 1]
 [-1]
 [ 1]] [[0]] 


[[-1]
 [-1]
 [ 0]
 [ 1]] [[-1]] 


[[ 1]
 [ 1]
 [-1]
 [ 0]] [[0]] 


[[0]
 [0]
 [0]
 [1]] [[0]] 


[[ 0]
 [ 1]
 [ 1]
 [-1]] [[1]] 


[[-1]
 [ 1]
 [-1]
 [ 1]] [[0]] 


[[-1]
 [ 0]
 [ 1]
 [ 0]] [[0]] 


[[1]
 [0]
 [0]
 [0]] [[1]] 


[[-1]
 [ 1]
 [-1]
 [ 1]] [[0]] 


[[ 0]
 [ 1]
 [-1]
 [ 0]] [[0]] 


[[ 0]
 [-1]
 [-1]
 [ 1]

[Something tricky here - we shouldn't have 1s and -1s - these should be between -127 and 127? (It's okay these should be zero but are to do with the integer division with non-integers.]

This works now.

In [215]:
vpu.forward_processing(rand_input)

array([[89.],
       [ 0.],
       [ 0.],
       [89.]])

In [216]:
rand_input.any()

True

In [217]:
a = rand_input
c = a*127//np.linalg.norm(a); c

array([[89.],
       [ 0.],
       [ 0.],
       [89.]])

In [218]:
np.dot(vpu.pi.eigenvector.T, c)

array([[11214.]])

In [219]:
vpu.forward_processing(np.ones(shape=(4,1)))

array([[63.],
       [63.],
       [63.],
       [63.]])

In [220]:
vpu.forward(np.ones(shape=(4,1)))

array([[1]], dtype=int8)

In [221]:
np.dot(vpu.pi.eigenvector.T, vpu.forward_processing(np.ones(shape=(4,1))))//127

array([[125.]])

In [222]:
r in [-1, 0, 1]

True

In [223]:
a = np.asarray([[-1],[0],[-1],[-1]]); a

array([[-1],
       [ 0],
       [-1],
       [-1]])

In [224]:
23 in [-1, 0, 1]

False

In [225]:
np.linalg.norm(a)

1.7320508075688772

In [226]:
b = a/np.linalg.norm(a); b

array([[-0.57735027],
       [ 0.        ],
       [-0.57735027],
       [-0.57735027]])

In [227]:
vpu.pi.eigenvector

array([[63],
       [63],
       [63],
       [63]], dtype=int8)

In [228]:
np.dot(vpu.pi.eigenvector.T, b)

array([[-109.11920088]])

In [229]:
vpu.forward(b)

array([[-1]], dtype=int8)

In [230]:
vpu.forward_processing(a)

array([[-74.],
       [  0.],
       [-74.],
       [-74.]])

In [231]:
a.any()

True

In [232]:
c = a*127//np.linalg.norm(a); c

array([[-74.],
       [  0.],
       [-74.],
       [-74.]])

In [233]:
np.dot(vpu.pi.eigenvector.T, c)//127

array([[-111.]])

In [234]:
size = 4
vpu = BinaryVPU(size)
for _ in range(0, 1000):
    vpu.update_cov(rand_same(negative=True, size=size), power_iterate=True)
print(vpu.pi.eigenvector)
try:
    for i in range(0, 100):
        rand_input = np.random.randint(low=-1, high=2, size=(size, 1))
        r = vpu.forward(rand_input)
        # assert r in [-1, 0, 1]
        print(rand_input, r, "\n\n")
except AssertionError:
    print(f"Error: {vpu}\n\nInput: {rand_input}\nR: {r}\n\n")

[[63]
 [63]
 [63]
 [63]]
[[-1]
 [-1]
 [-1]
 [ 1]] [[0]] 


[[-1]
 [ 0]
 [ 0]
 [ 1]] [[0]] 


[[1]
 [0]
 [0]
 [0]] [[0]] 


[[ 1]
 [-1]
 [ 0]
 [-1]] [[0]] 


[[-1]
 [-1]
 [-1]
 [ 1]] [[-1]] 


[[ 0]
 [-1]
 [ 1]
 [ 0]] [[0]] 


[[ 1]
 [ 0]
 [ 1]
 [-1]] [[0]] 


[[-1]
 [ 1]
 [ 0]
 [ 1]] [[0]] 


[[0]
 [0]
 [1]
 [0]] [[1]] 


[[ 0]
 [ 1]
 [ 0]
 [-1]] [[0]] 


[[1]
 [1]
 [1]
 [1]] [[1]] 


[[ 0]
 [-1]
 [ 0]
 [ 1]] [[0]] 


[[ 1]
 [ 0]
 [-1]
 [-1]] [[0]] 


[[-1]
 [ 0]
 [ 0]
 [-1]] [[-1]] 


[[-1]
 [-1]
 [-1]
 [-1]] [[-1]] 


[[0]
 [0]
 [1]
 [0]] [[0]] 


[[0]
 [0]
 [0]
 [1]] [[1]] 


[[-1]
 [ 0]
 [ 0]
 [ 1]] [[0]] 


[[1]
 [1]
 [0]
 [1]] [[1]] 


[[-1]
 [-1]
 [-1]
 [ 0]] [[-1]] 


[[0]
 [1]
 [1]
 [0]] [[1]] 


[[ 0]
 [ 1]
 [ 1]
 [-1]] [[0]] 


[[-1]
 [-1]
 [ 0]
 [ 0]] [[-1]] 


[[-1]
 [-1]
 [ 1]
 [-1]] [[-1]] 


[[ 0]
 [ 1]
 [ 0]
 [-1]] [[0]] 


[[ 0]
 [ 1]
 [-1]
 [-1]] [[0]] 


[[-1]
 [ 1]
 [ 0]
 [ 0]] [[0]] 


[[-1]
 [ 0]
 [-1]
 [ 1]] [[0]] 


[[ 1]
 [-1]
 [-1]
 [ 0]] [[0]] 


[[ 0]
 [-1]

In [235]:
r.dtype

dtype('int8')

## Todo

* Put the above as tests.
* Work on the predictions / feedback processing.

If we don't PBT going forwards, we get different values up to +/-1/127 indicating +ve and -ve matches. Going backwards we lose that information - we just have r is +/- 1. But the strength of belief in r could be indicated by r over time - e.g. intermittent r is ~ 63/127 which means "maybe feature".

This suggests some buffering of r over time on the feedback? Or just max out r so when +/-1 output full ev but when 0 output nothing - leaving it to the error?

In [ ]:
print(r)

In [ ]:
    def r_forward_processing(self, r_forward):
        """Scale r to -1 to 1 and PBT."""
        # Threshold r
        binary_values = ternary_pbt(r_forard)
        return binary_values

    def r_backward_processing(self, r_backward):
        """Rescale r to -L/sqrt(L) to L/sqrt(L) and PBT."""
        # Scale to ternary
        scaled_r = r_backward*self.scale_backward
        return scaled_r

    def pred_input_processing(self, pred_inputs):
        """Apply PBT to get outputs in range -1, 0, 1."""
        binary_values = ternary_pbt(pred_inputs)
        return binary_values